In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from time import sleep
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import random

In [2]:
first_plusword_num = 14998
first_plusword_date = '2022-05-22'
first_plusword_date = datetime.fromisoformat(first_plusword_date)
df_main = pd.read_csv('main.csv')
df_main['puzzle_number'] = df_main['puzzle_number'].astype('int')
done = df_main['puzzle_number'].values.tolist()
done

[3, 4, 6, 1, 2, 7]

In [3]:
url ='https://www.telegraph.co.uk/puzzles/puzzles/?source=archive#plusword/plusword/plusword-19952'
driver = webdriver.Firefox()
driver.get(url)
sleep(2)
email = 'tomwhittingtont@gmail.com'
password = '7P7gp8pMu86ycUV'
driver.switch_to.frame('sp_message_iframe_784380')
#driver = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, '/html/body/div/div[2]/div[3]/div/div[2]/div[2]/button[1]'))).click()
driver.find_element(By.XPATH, '/html/body/div/div[2]/div[3]/div/div[2]/div[2]/button[1]').click()
sleep(2)
driver.refresh()
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/p[2]/a').click()
sleep(2)
driver.find_element(By.ID, "email").send_keys(email)
#username_input.
driver.find_element(By.ID, 'login-button').click()
sleep(2)
driver.find_element(By.ID, "password").send_keys(password)
driver.find_element(By.ID, 'login-button').click()
sleep(2)

In [4]:
url_stem = 'https://www.telegraph.co.uk/puzzles/puzzles/?source=archive#plusword/plusword/plusword-'

In [5]:
url_list =[]
for _ in range(0, 30):
    plusword_num = random.randint(0,30)
    
    if plusword_num not in done:
        url = url_stem + str(first_plusword_num + plusword_num)
        url_list.append(url)

row_list =[]

for url in url_list:
    
    try:
        driver.get(url)
        sleep(2)
        driver.switch_to.frame('gameplayiframe')
        sleep(5)
        driver.find_element(By.XPATH, '/html/body/div[2]/div[9]/header/button').click()
        
        
    except:
        pass
    
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/button').click()
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/div[2]/ul/li[3]/button').click()
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/div[1]/ul/li[1]/button').click()
        
        row={}
        yellow = []
        green = []
        answer=''

        puzzle_number = driver.find_elements(By.CLASS_NAME, "hangboard")
        puzzle_number = puzzle_number[0].find_element(By. TAG_NAME, 'h1')
        puzzle_number = puzzle_number.text

        puzzle_number=int(''.join(str(number) for number in re.findall(r'\d+', puzzle_number)))

        plusword_date = first_plusword_date +  timedelta(days=puzzle_number)

        plusword_date =plusword_date.strftime("%Y-%m-%d")

        row.update({'date' : plusword_date, 'puzzle_number' : str(puzzle_number)})

        clues = driver.find_elements(By.CLASS_NAME, "clue__text")


        clue_num = 0
        for clue in clues[0:5]:
            clue_num +=1
            row.update({'clue_across_' + str(clue_num) :clue.text})

        clue_num = 0
        for clue in clues[5:10]:
            clue_num +=1
            row.update({'clue_down_' + str(clue_num) :clue.text})



        for table_row in driver.find_elements(By.CLASS_NAME, "row"):
            for cell in table_row.find_elements(By.TAG_NAME, 'td'):
                letter = cell.find_element(By.CLASS_NAME, 'letter')
                answer = answer + letter.text

                cell_class = cell.get_attribute("class")

                if 'right-letter-wrong-column' in cell_class:
                    yellow.append((int(re.search('C\d*', cell_class).group(0).strip('C'))) +1)

                if 'right-letter-right-column' in cell_class:
                    green.append((int(re.search('C\d*', cell_class).group(0).strip('C'))) +1 )

        row.update({'plusword_solution' : answer[-5:]})

        for answer_num in range(1,6):
            row.update({'answer_' + str(answer_num) : answer[5*(answer_num-1):(5*answer_num)]})
        row.update({'yellow' : yellow, 'green' : green})
        
        row_list.append(row)
        sleep(5)

row_list

df = pd.DataFrame.from_records([row])

df

In [6]:
row_list =[]
puzzle_number_list=[]
plusword_date_list = []
yello_list = []
green_list =[]
my_dict= {'date' : [],
          'puzzle_number' : [],
          'plusword_solution' : [],
          'clue_across_1' : [],
          'clue_across_2' : [],
          'clue_across_3' : [],
          'clue_across_4' : [],
          'clue_across_5' : [],
          'clue_down_1' : [],
          'clue_down_2' : [],
          'clue_down_3' : [],
          'clue_down_4' : [],
          'clue_down_5' : [],
          'answer_1' : [],
          'answer_2' : [],
          'answer_3' : [],
          'answer_4' : [],
          'answer_5' : [],
          'yellow' : [],
          'green' : []
         }

for url in url_list:
    
    try:

        driver.get(url)
        sleep(15)
        driver.refresh()
    except:
        pass
    
    try:
        driver.switch_to.frame('gameplayiframe')
        sleep(5)
    except:
        pass
    
    try:
        driver.find_element(By.XPATH, '/html/body/div[2]/div[9]/header/button').click()
    except:
        pass
        


    try:
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/button').click()
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/div[2]/ul/li[3]/button').click()
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/header/ul[2]/li[1]/div[1]/ul/li[1]/button').click()

        yellow = []
        green = []
        answer=''

        puzzle_number = driver.find_elements(By.CLASS_NAME, "hangboard")
        puzzle_number = puzzle_number[0].find_element(By. TAG_NAME, 'h1')
        puzzle_number = puzzle_number.text
        puzzle_number =int(''.join(str(number) for number in re.findall(r'\d+', puzzle_number)))

        plusword_date = first_plusword_date +  timedelta(days=puzzle_number)

        plusword_date=plusword_date.strftime("%Y-%m-%d")

        my_dict['date'].append(plusword_date)
        my_dict['puzzle_number'].append(str(puzzle_number))

        clues = driver.find_elements(By.CLASS_NAME, "clue__text")


        clue_num = 0
        for clue in clues[0:5]:
            clue_num +=1
            my_dict['clue_across_' + str(clue_num)].append(clue.text)

        clue_num = 0
        for clue in clues[5:10]:
            clue_num +=1
            my_dict['clue_down_' + str(clue_num)].append(clue.text)



        for table_row in driver.find_elements(By.CLASS_NAME, "row"):
            for cell in table_row.find_elements(By.TAG_NAME, 'td'):
                letter = cell.find_element(By.CLASS_NAME, 'letter')
                answer = answer + letter.text

                cell_class = cell.get_attribute("class")

                if 'right-letter-wrong-column' in cell_class:
                    yellow.append((int(re.search('C\d*', cell_class).group(0).strip('C'))) +1)

                if 'right-letter-right-column' in cell_class:
                    green.append((int(re.search('C\d*', cell_class).group(0).strip('C'))) +1 )

        my_dict['plusword_solution'].append(answer[-5:])

        for answer_num in range(1,6):
                my_dict['answer_' + str(answer_num)].append(answer[5*(answer_num-1):(5*answer_num)])
        my_dict['yellow'].append(yellow)
        my_dict['green'].append(green)

        print(my_dict)

        df = pd.DataFrame.from_dict(my_dict)
        df_main = pd.concat([df_main, df])
        df_main = df_main.drop_duplicates(subset=['date'])
        df_main.to_csv('main.csv', index=False)


        sleep(5)
        
    except:
        pass

In [7]:
my_dict

{'date': [],
 'puzzle_number': [],
 'plusword_solution': [],
 'clue_across_1': [],
 'clue_across_2': [],
 'clue_across_3': [],
 'clue_across_4': [],
 'clue_across_5': [],
 'clue_down_1': [],
 'clue_down_2': [],
 'clue_down_3': [],
 'clue_down_4': [],
 'clue_down_5': [],
 'answer_1': [],
 'answer_2': [],
 'answer_3': [],
 'answer_4': [],
 'answer_5': [],
 'yellow': [],
 'green': []}

In [8]:
df_main

,date,puzzle_number,plusword_solution,clue_across_1,clue_across_2,clue_across_3,clue_across_4,clue_across_5,clue_down_1,clue_down_2,...,clue_down_4,clue_down_5,answer_1,answer_2,answer_3,answer_4,answer_5,yellow,green,Unnamed: 20
0,2022-05-26,3,SHIRK,Type of clay-pigeon shooting,Grub,Criminal slang,There's no ____ without fire,Person finding something abhorrent,Cut,Fate,...,A disease of grasses,Summon (up),Spud,SKEET,LARVA,ARGOT,SMOKE,HATER,"[2, 8, 12, 19, 21, 25]","[1, 16]"
1,2022-05-27,4,FLAME,Out of order,Division of a long poem,Extrapolate,More disastrous,Spine-chilling,Substances with a pH of less than 7,"____ Miner, 1980s video game",...,Young ox,Apologies!,AMISS,CANTO,INFER,DIRER,SCARY,"[1, 2, 7, 13, 14, 19]",[23],NaN
2,2022-05-29,6,ELFIN,Road of a type made famous by Las Vegas,Raita (anag.),Peripheral,Lose consciousness,Material used for cleaning teeth,Cane,Relating to the motion of the sea,...,"Jeremy ____, Brideshead Revisited actor",Total rubbish!,STRIP,TIARA,ADDON,FAINT,FLOSS,"[7, 16, 18, 19, 21]","[4, 15, 22]",NaN
3,2022-05-24,1,LURID,The ship of the desert,Broadest below the middle,Type of coffee,Bogs,Dote on,Punctuation mark,Eschew,...,Clear air,Charter,CAMEL,OVATE,MOCHA,MIRES,ADORE,"[5, 17, 22, 24]",[18],NaN
4,2022-05-25,2,THUMB,Spicy sauce,Troy,Bone in the lower leg,Lends a hand,Exasperates,Stringed instrument,The fact of being elsewhere,...,Set of furniture or rooms,Accumulate,SALSA,ILIUM,TIBIA,ABETS,RILES,"[9, 10, 13, 17, 19]",[11],NaN
5,2022-05-30,7,MIGHT,European country,Verboten,Ruby type in printing,Music genre,Recruit by force,Imprint,Bleeper,...,Small amounts,"Fielding and Gallagher, e.g.",SPAIN,TABOO,AGATE,METAL,PRESS,"[4, 6, 12, 14, 18]",[16],NaN
